# Multi-Robot Omnidirectional Go2Goal

In [ ]:
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.notebook import tqdm

from dimp.robots import (
    PointMassInput, PointMassRobot, PointMassState, MPCData
)

from dimp.utils import init_matplotlib, get_colors, display_animation, MultiRobotArtistFlags

init_matplotlib()

In [ ]:
np.random.seed(0)
torch.manual_seed(0)

### Create the Data for the MPC

In [ ]:
nr = 3      # number of robots

ns = PointMassState.n   # number of states (x, y, vx, vy)
ni = PointMassInput.n   # number of inputs (ax, ay)

nc = 10      # number of control intervals

s0 = [cp.Parameter(ns, name=f"s0_{i}") for i in range(nr)]  # initial state

# Note that class MPCData does initialize all the cp variables and parameters.
robots_data = [MPCData(
    nc=nc,
    states_list=[PointMassState(s0[i])] + [PointMassState(cp.Variable(ns, name=f"s{i}_{k+1}")) for k in range(nc)],
    statesbar_list=[PointMassState(s0[i])] + [PointMassState(cp.Parameter(ns)) for _ in range(nc)],
    inputs_list=[PointMassInput(cp.Variable(ni, name=f"u{i}_{k}")) for k in range(nc)],
    inputsbar_list=[PointMassInput(cp.Parameter(ni)) for _ in range(nc)],
) for i in range(nr)]

dt = 0.1    # sampling time

rob_mass = [1.0 for _ in range(nr)]     # robot masses

goals = [cp.Parameter(2, name=f"goal_{i}") for i in range(nr)]

### Create the MPC Problem

In [ ]:
mpc_params = {
    'k_forces': cp.Parameter(nonneg=True, name="k_forces"),
    'k_vel': cp.Parameter(nonneg=True, name="k_vel"),
    'k_terminal_pos': cp.Parameter(nonneg=True, name="k_terminal_pos"),
    'k_terminal_vel': cp.Parameter(nonneg=True, name="k_terminal_vel"),
}

In [ ]:
def generate_constraints():
    # p_k+1 - (p_k + v_k*dt) == 0
    # v_k+1 - (v_k + u_k*dt + fc_k*dt) == 0 
    rob_dyn_constraints = [[
        cp.hstack([
            robots_data[i].xi[k+1] - (robots_data[i].xi[k] + robots_data[i].vxi[k]*dt),
            robots_data[i].yi[k+1] - (robots_data[i].yi[k] + robots_data[i].vyi[k]*dt),
            robots_data[i].vxi[k+1] - (robots_data[i].vxi[k] + (robots_data[i].fxi[k]) / rob_mass[i] * dt),
            robots_data[i].vyi[k+1] - (robots_data[i].vyi[k] + (robots_data[i].fyi[k]) / rob_mass[i] * dt),
        ]) == 0
        for k in range(nc)
    ] for i in range(nr)]
    
    constraints = sum([
        sum(rob_dyn_constraints, []),
    ], [])
    
    return constraints

_ = generate_constraints()

State: $s_{i,k} = \begin{bmatrix} p_{i,k}^T & v_{i,k}^T \end{bmatrix}^T$

Input: $u_{i,k} = f_{i,k}$

Parameters: $\theta = \begin{bmatrix} k_f & k_v & k_{fin, p} & k_{fin, v} \end{bmatrix}$

$$
\begin{align}
& \min_{s, u} & \sum_{i=0}^{n_r} \left( \sum_{k=0}^{n_c} 
\left( \left\lVert p_{i,k} - p_{g,i} \right\rVert_Q^2 
    + k_f \left\lVert u_{i,k} \right\rVert_R^2
    + k_v \left\lVert v_{i,k} \right\rVert_R^2 \right) \\
    + k_{fin, p} \left\lVert p_{i,n_c} - p_{g,i} \right\rVert_Q^2
    + k_{fin, v} \left\lVert v_{i,n_c} \right\rVert_Q^2 \right) \\
& \text{s.t.} \quad & \text{dynamics} \\
& & \text{input limits}
\end{align}
$$

In [ ]:
def create_qp():
    objective = cp.Minimize(
        sum([
            sum([
                cp.sum_squares(cp.hstack([
                    robots_data[i].xi[k] - goals[i][0],
                    robots_data[i].yi[k] - goals[i][1],
                ]))
                for k in range(nc)
            ])
            for i in range(nr)
        ]) \
        + mpc_params['k_forces'] * cp.sum(cp.hstack([
            cp.sum_squares(robots_data[i].fx) + cp.sum_squares(robots_data[i].fy)
        for i in range(nr)])) \
        + mpc_params['k_vel'] * cp.sum(cp.hstack([
            cp.sum_squares(robots_data[i].vxi[k]) + cp.sum_squares(robots_data[i].vyi[k])
        for i in range(nr) for k in range(1, nc+1)])) \
        + mpc_params['k_terminal_pos'] * cp.sum(cp.hstack([
            cp.sum_squares(robots_data[i].xi[-1] - goals[i][0].value) + cp.sum_squares(robots_data[i].yi[-1] - goals[i][1].value)
        for i in range(nr)])) \
        + mpc_params['k_terminal_vel'] * cp.sum(cp.hstack([
            cp.sum_squares(robots_data[i].vxi[-1]) + cp.sum_squares(robots_data[i].vyi[-1])
        for i in range(nr)]))
    )
    
    constraints = generate_constraints()
    
    problem = cp.Problem(objective, constraints)
    assert problem.is_dpp()
    
    return problem

In [ ]:
def simulate(steps: int):
    s0[0].value = np.array([0.0, 0.0, 0.0, 0.0])
    s0[1].value = np.array([1.0, 0.0, 0.0, 0.0])
    s0[2].value = np.array([1.0, 0.0, 0.0, 0.0])
    goals[0].value = np.array([1.0, 1.0])
    goals[1].value = np.array([2.0, 0.0])
    goals[2].value = np.array([3.0, 1.0])
    
    mpc_params['k_forces'].value = 1.0
    mpc_params['k_vel'].value = 1.0
    mpc_params['k_terminal_pos'].value = 1.0
    mpc_params['k_terminal_vel'].value = 1.0
    
    problem = create_qp()
    
    trajectories = [np.zeros((steps+1, ns)) for _ in range(nr)]
    for i in range(nr):
        trajectories[i][0, :] = s0[i].value
    inputs = [np.zeros((steps, ni)) for _ in range(nr)]
    
    for k in range(steps):
        problem.solve()
        
        s0[0].value = robots_data[0].statei[1].value
        s0[1].value = robots_data[1].statei[1].value
        
        for i in range(nr):
            trajectories[i][k+1, :] = s0[i].value
            inputs[i][k, :] = robots_data[i].inputi[0].value
            
    return trajectories, inputs

In [ ]:
def plot_traj(trajectories):
    plt.figure()
    
    for i in range(nr):
        plt.plot(
            trajectories[i][:, 0],
            trajectories[i][:, 1],
            label=f"Robot {i+1} Traj",
        )
    
        plt.scatter(
            goals[i].value[0],
            goals[i].value[1],
            marker="x",
            s=50,
            label=f"Robot {i+1} Goal",
        )
    
    plt.xlabel("x [m]")
    plt.ylabel("y [m]")
    plt.title("Robot Trajectories")
    plt.legend()
    plt.show()

In [ ]:

def traj_2_s_history(trajectories) -> list[list[list[np.ndarray]]]:
    nc = 2                      # number of robot classes (unicycle, omnidirectional)
    nk = len(trajectories[0])   # timesteps
    s_history = [None for _ in range(nk)]
    for k in range(nk):
 
        s_history[k] = [
            [
                (trajectories[i][k, 0:3] if c == 0 else trajectories[i][k, 0:2])
                for i in (range(nr) if c == 1 else [])
            ]
            for c in range(nc)
        ]
    
    return s_history

In [ ]:
trajectories, inputs = simulate(steps=100)
plot_traj(trajectories)

In [ ]:
plt.rc('text', usetex=False)

s_history = traj_2_s_history(trajectories)

mra_flags = MultiRobotArtistFlags()
mra_flags.centroid = False
mra_flags.goals = True
mra_flags.obstacles = True
mra_flags.voronoi = False

display_animation(
    s_history=s_history,
    goals=[goal.value for goal in goals],
    obstacles=[],
    dt=0.1,
    method='html',
    x_lim=[-5, 5],
    y_lim=[-5, 5],
    flags=mra_flags,
)

### Create the CvxpyLayer

In [ ]:
cvxpylayer = CvxpyLayer(
    problem=create_qp(),
    parameters=s0 + goals + list(mpc_params.values()),
    variables=sum([[
        robots_data[i].statei[k] for k in range(1, nc + 1)
    ] for i in range(nr)], []) + sum([[
        robots_data[i].inputi[k] for k in range(nc)
    ] for i in range(nr)], []),
)

s0_th = [torch.tensor(s0[i].value, dtype=torch.float32) for i in range(nr)]
goals_th = [torch.tensor(goals[i].value, dtype=torch.float32) for i in range(nr)]
kf_th = torch.nn.Parameter(torch.tensor([1.0]))
vel_th = torch.nn.Parameter(torch.tensor([1.0]))
terminal_pos_th = torch.nn.Parameter(torch.tensor([1.0]))
terminal_vel_th = torch.nn.Parameter(torch.tensor([1.0]))
mpc_params_th = {
    'k_forces': kf_th,
    'k_vel': vel_th,
    'k_terminal_pos': terminal_pos_th,
    'k_terminal_vel': terminal_vel_th,
}

params_torch = []
optim = torch.optim.Adam(list(mpc_params_th.values()), lr=5e-2)

solution = cvxpylayer(*s0_th, *goals_th, *list(mpc_params_th.values()))

solution[0].sum().backward()

In [ ]:
def task_loss(states, inputs):
    loss = torch.square(
        states[:, :, 0:2] - torch.tensor([goal.value for goal in goals]).view(nr, 1, 2)
    ).sum() + torch.square(inputs).sum()
    
    return loss

In [ ]:
def rollout(cvxpylayer, s0_th, other_params, steps: int):
    states = torch.zeros((nr, steps+1, ns))
    inputs = torch.zeros((nr, steps, ni))
    
    for i in range(nr):
        states[i, 0, :] = s0_th[i]
    
    for k in range(steps):
        sol = cvxpylayer(*s0_th, *other_params)
        
        # Extract states and inputs
        for i in range(nr):
            states[i, k+1, :] = sol[nc*i]
            inputs[i, k, :] = sol[nc*nr + nc*i]
        
        # Update s0_th for next iteration
        for i in range(nr):
            s0_th[i] = sol[nc*i]
            
    return states, inputs

In [ ]:
n_epochs = 100
history = []

with torch.no_grad():
    for i in range(nr):
        s0_th[i].copy_(torch.tensor(s0[i].value, dtype=torch.float32))
        goals_th[i].copy_(torch.tensor(goals[i].value, dtype=torch.float32))
        kf_th.copy_(torch.tensor([1.0]))
        
with tqdm(range(n_epochs)) as pbar:
    for epoch in range(n_epochs):
        pbar.update(1)
        
        optim.zero_grad()
        
        s0_min = torch.tensor([-10.0, -10.0, -1.0, -1.0])
        s0_max = torch.tensor([10.0, 10.0, 1.0, 1.0])
        s0_th = [torch.rand(4, dtype=torch.float32) * (s0_max - s0_min) + s0_min for _ in range(nr)]
        
        states, inputs = rollout(
            cvxpylayer, s0_th, goals_th + list(mpc_params_th.values()), steps=50,
        )
        
        loss = task_loss(states, inputs)
        loss.backward()
        
        optim.step()
        
        history.append({
            **{key: param.item() for key, param in mpc_params_th.items()},
            'loss': loss.item(),
        })

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 5))

for i in range(4):
    key = list(mpc_params.keys())[i]
    
    axs[i//2, i%2].plot(
        [h[key] for h in history],
    )
    axs[i//2, i%2].set(
        xlabel="Epoch",
        ylabel=f"{key}",
        title=f"Evolution of {key} during training"
    )

plt.show()